### portmy database: stocks table
### stock database: buy, price tables

In [2]:
import pandas as pd
import numpy as np
import os
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)

data_path = os.path.join(os.path.expanduser("~"), "OneDrive","A5","Data")
csv_path = os.path.join(os.path.expanduser("~"), "iCloudDrive")
one_path = os.path.join(os.path.expanduser("~"), "OneDrive","Documents","Data")
osd_path = os.path.join(os.path.expanduser("~"),"OneDrive","Documents","obsidian-git-sync","Data")
dts_path = os.path.join(os.path.expanduser("~"),"Downloads","Datasets")

today = date.today()
today

datetime.date(2025, 1, 19)

In [3]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2025-01-19
yesterday: 2025-01-17 00:00:00


In [4]:
yesterday = yesterday.date()
week_ago = yesterday -timedelta(days = 8)
print(f'a week ago: {week_ago}')
print(f'yesterday: {yesterday}')

a week ago: 2025-01-09
yesterday: 2025-01-17


### Restart & Run All Cells

### This process affects only already in port stocks. To highlight price changes in percent.

In [7]:
cols = 'name price_w price_d percent perform mrkt '.split()

format_dict = {
    'qty':'{:,}',    
    'price':'{:.2f}','price_d':'{:.2f}','price_w':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'maxp':'{:.2f}','minp':'{:.2f}','opnp':'{:.2f}',    
    'pct':'{:,.2f}%','percent':'{:,.2f}%',
    
    'pe':'{:.2f}','pbv':'{:.2f}',
    'paid_up':'{:,.2f}','market_cap':'{:,.2f}',
    'daily_volume':'{:,.2f}','beta':'{:,.2f}', 
    'dly_vol':'{:,.2f}', 
}

In [8]:
sql = """
SELECT name, date, price 
FROM buy 
ORDER BY date DESC 
LIMIT 1
"""
df_buy = pd.read_sql(sql, const)
df_buy.style.format(format_dict)

,name,date,price
0,TVO,2024-11-06,24.90


In [9]:
names = df_buy['name']
name = names.to_string(index=False)

sql = '''
SELECT * FROM stocks WHERE name = '%s'
'''
sql = sql % name
print(sql)

df_stocks = pd.read_sql(sql, conmy)
df_stocks.style.format(format_dict) 


SELECT * FROM stocks WHERE name = 'TVO'



,id,name,market,price,max_price,min_price,pe,pbv,paid_up,market_cap,daily_volume,beta,ticker_id,created_at,updated_at
0,585,TVO,SETTHSI / SETWB,29.00,31.25,26.50,11.18,2.40,889.47,"25,794.67",18.84,0.43,585,2017-07-23 07:25:46.640728,2023-01-31 03:08:01.449850


In [10]:
sql = '''
SELECT * FROM price WHERE name = "%s" ORDER BY date DESC LIMIT 5
'''
sql = sql % name
print(sql)

df_price = pd.read_sql(sql, const)
df_price.style.format(format_dict)


SELECT * FROM price WHERE name = "TVO" ORDER BY date DESC LIMIT 5



,name,date,price,maxp,minp,qty,opnp
0,TVO,2025-01-17,21.30,21.40,21.10,"167,391",21.40
1,TVO,2025-01-16,21.30,21.70,21.30,"222,261",21.70
2,TVO,2025-01-15,21.50,21.70,21.50,"231,252",21.70
3,TVO,2025-01-14,21.50,21.80,21.50,"266,383",21.50
4,TVO,2025-01-13,21.50,21.80,21.50,"214,558",21.60


In [11]:
sql = '''
SELECT name
FROM buy 
WHERE active = 1
ORDER BY name'''
df_price = pd.read_sql(sql, const)

names = df_price.name.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))

sql = """
SELECT name, price AS price_d 
FROM price 
WHERE date = '%s' AND name IN (%s)
ORDER BY name, date"""
sql = sql % (yesterday, in_p)
print(sql)

df_today = pd.read_sql(sql, const)
df_today.shape[0]


SELECT name, price AS price_d 
FROM price 
WHERE date = '2025-01-17' AND name IN ('3BBIF', 'AH', 'AIMIRT', 'ASK', 'ASP', 'AWC', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'IVL', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'ORI', 'PTG', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'TOA', 'TVO', 'WHAIR', 'WHART')
ORDER BY name, date


31

In [12]:
sql = """
SELECT name, price AS price_w
FROM price 
WHERE date = '%s' AND name IN (%s) 
ORDER BY name"""
sql = sql % (week_ago, in_p)
print(sql)

df_wkago = pd.read_sql(sql, const)
df_wkago.shape[0]


SELECT name, price AS price_w
FROM price 
WHERE date = '2025-01-09' AND name IN ('3BBIF', 'AH', 'AIMIRT', 'ASK', 'ASP', 'AWC', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'IVL', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'ORI', 'PTG', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'TOA', 'TVO', 'WHAIR', 'WHART') 
ORDER BY name


31

In [13]:
trend = pd.merge(df_today, df_wkago, on='name',how='inner')
trend.shape

(31, 3)

In [14]:
def performance(vals):
    price_d, price_w = vals
    if (price_d > price_w):
        return 'Better'
    elif (price_d < price_w):
        return 'Worse'
    else:
        return 'No Change'

In [15]:
trend['percent'] = (trend.price_d-trend.price_w)/trend.price_w * 100

In [16]:
trend

,name,price_d,price_w,percent
0,3BBIF,5.10,5.10,0.00
1,AH,15.00,15.10,-0.66
2,AIMIRT,10.50,10.50,0.00
3,ASK,8.45,9.75,-13.33
4,ASP,2.32,2.30,0.87
5,AWC,3.14,3.36,-6.55
6,BCH,14.50,14.80,-2.03
7,CPNREIT,12.20,12.10,0.83
8,DIF,8.30,8.40,-1.19
9,GVREIT,6.65,6.50,2.31


In [17]:
trend["perform"] = trend[["price_d","price_w"]].apply(performance, axis=1)

In [18]:
trend.sort_values(by=['percent'],ascending=[True]).style.format(format_dict)

,name,price_d,price_w,percent,perform
3,ASK,8.45,9.75,-13.33%,Worse
27,TOA,12.90,14.50,-11.03%,Worse
24,SYNEX,13.10,14.30,-8.39%,Worse
18,PTTGC,21.50,23.30,-7.73%,Worse
19,RCL,25.00,27.00,-7.41%,Worse
5,AWC,3.14,3.36,-6.55%,Worse
13,KCE,21.70,23.00,-5.65%,Worse
22,SINGER,6.65,7.00,-5.00%,Worse
11,JMART,10.90,11.20,-2.68%,Worse
17,PTG,7.65,7.85,-2.55%,Worse


In [19]:
file_name = 'trend.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
osd_file = osd_path + file_name

trend.sort_values(['percent'],ascending=[True]).to_csv(output_file, index=False)
trend.sort_values(['percent'],ascending=[True]).to_csv(data_file, index=False)
trend.sort_values(['percent'],ascending=[True]).to_csv(osd_file, index=False)

### Filter only performance = "Worse"

In [21]:
mask = trend.perform == 'Worse'
trend[mask]

,name,price_d,price_w,percent,perform
1,AH,15.00,15.10,-0.66,Worse
3,ASK,8.45,9.75,-13.33,Worse
5,AWC,3.14,3.36,-6.55,Worse
6,BCH,14.50,14.80,-2.03,Worse
8,DIF,8.30,8.40,-1.19,Worse
11,JMART,10.90,11.20,-2.68,Worse
12,JMT,14.40,14.70,-2.04,Worse
13,KCE,21.70,23.00,-5.65,Worse
14,MCS,6.35,6.40,-0.78,Worse
16,ORI,3.34,3.40,-1.76,Worse


In [22]:
trend.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,proportion
perform,
Worse,67.74%
Better,19.35%
No Change,12.90%


In [23]:
sql = '''
SELECT name, max_price AS max, min_price AS min, pe, pbv, daily_volume AS volume, beta, market
FROM stocks
'''
my_stocks = pd.read_sql(sql, conmy)
my_stocks.shape

(246, 8)

In [24]:
filters = [
   (my_stocks.market.str.contains('SET50')),
   (my_stocks.market.str.contains('SET100')),
   (my_stocks.market.str.contains('mai'))]
values = ['SET50','SET100','mai']
my_stocks["mrkt"] = np.select(filters, values, default='SET999')

In [25]:
trend2 = pd.merge(trend, my_stocks, on='name', how='inner')
trend2.sort_values(['percent'],ascending=[True]).shape

(31, 13)

In [26]:
set50 = trend2.mrkt.str.contains('SET50') 
flt_set50 = trend2[set50]
flt_set50[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt
18,PTTGC,23.30,21.50,-7.73%,Worse,SET50
5,AWC,3.36,3.14,-6.55%,Worse,SET50
11,JMART,11.20,10.90,-2.68%,Worse,SET50
12,JMT,14.70,14.40,-2.04%,Worse,SET50
20,SCC,154.00,156.00,1.30%,Better,SET50
10,IVL,21.90,23.60,7.76%,Better,SET50


In [27]:
flt_set50.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,proportion
perform,
Worse,66.67%
Better,33.33%


In [28]:
set100 = trend2.mrkt.str.contains('SET100')
flt_set100 = trend2[set100]
flt_set100[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt
19,RCL,27.00,25.00,-7.41%,Worse,SET100
13,KCE,23.00,21.70,-5.65%,Worse,SET100
22,SINGER,7.00,6.65,-5.00%,Worse,SET100
17,PTG,7.85,7.65,-2.55%,Worse,SET100
6,BCH,14.80,14.50,-2.03%,Worse,SET100
16,ORI,3.40,3.34,-1.76%,Worse,SET100
23,STA,17.00,16.80,-1.18%,Worse,SET100


In [29]:
flt_set100[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,proportion
perform,
Worse,100.00%


In [30]:
set999 = trend2.mrkt.str.contains('SET999')
flt_set999 = trend2[set999]
flt_set999[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt
3,ASK,9.75,8.45,-13.33%,Worse,SET999
27,TOA,14.50,12.90,-11.03%,Worse,SET999
24,SYNEX,14.30,13.10,-8.39%,Worse,SET999
25,TFFIF,6.40,6.25,-2.34%,Worse,SET999
28,TVO,21.80,21.30,-2.29%,Worse,SET999
26,TMT,3.02,2.96,-1.99%,Worse,SET999
8,DIF,8.40,8.30,-1.19%,Worse,SET999
14,MCS,6.40,6.35,-0.78%,Worse,SET999
29,WHAIR,6.50,6.45,-0.77%,Worse,SET999
1,AH,15.10,15.00,-0.66%,Worse,SET999


In [31]:
flt_set999[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,proportion
perform,
Worse,55.56%
No Change,22.22%
Better,22.22%


In [32]:
setmai = trend2.mrkt.str.contains('mai')
flt_setmai = trend2[setmai]
flt_setmai[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt


In [33]:
flt_setmai[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,proportion
perform,


In [34]:
const.close()
conmy.close()